In [ ]:
import re
from datetime import datetime
from pymongo import MongoClient
import mysql.connector
from mysql.connector import Error

email_pattern = re.compile(r'From\s+([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)')
date_pattern = re.compile(r'Date:\s+([A-Za-z]{3},\s+\d{1,2}\s+[A-Za-z]{3}\s+\d{4}\s+\d{2}:\d{2}:\d{2}\s+[-+]\d{4})')

with open('mbox.txt', 'r') as file:
    log_content = file.read()

emails = email_pattern.findall(log_content)
dates = date_pattern.findall(log_content)

structured_data = []

for email, date_str in zip(emails, dates):
    date_obj = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S %z')
    formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
    structured_data.append({'email': email,'date': formatted_date})

client = MongoClient('mongodb://localhost:27017/') # Replace with your MongoDB connection string
db = client['guvi']  # Replace with your database name
collection = db['user_history']  # Collection name

if structured_data:
    collection.insert_many(structured_data)
    print(f"Inserted {len(structured_data)} records into the 'user_history' collection.")
else:
    print("No data to insert.")

client.close()

def fetch_data_from_mongodb():
    try:
        client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection string
        db = client['database']  # Replace with your database name
        collection = db['user_history']  # Collection name

        documents = collection.find({}, {'_id': 0})  
        data = list(documents)  

        print(f"Fetched {len(data)} records from MongoDB.")
        return data

    except Exception as e:
        print(f"Error fetching data from MongoDB: {e}")
        return []

    finally:
        client.close()

def insert_data_into_mysql(data):
    try:
        connection = mysql.connector.connect(
            host='localhost',  # Replace with your MySQL host
            user='root',       # Replace with your MySQL username
            password='password',  # Replace with your MySQL password
            database='database'  # Replace with your MySQL database name
        )

        if connection.is_connected():
            cursor = connection.cursor()

            create_table_query = """
            CREATE TABLE IF NOT EXISTS user_history (
                id INT AUTO_INCREMENT PRIMARY KEY,
                email VARCHAR(255) NOT NULL,
                date DATETIME NOT NULL
            )
            """
            cursor.execute(create_table_query)
            print("Created or verified the 'user_history' table.")

            insert_query = """
            INSERT INTO user_history (email, date)
            VALUES (%s, %s)
            """
            records_to_insert = [(record['email'], record['date']) for record in data]

            cursor.executemany(insert_query, records_to_insert)
            connection.commit()
            print(f"Inserted {cursor.rowcount} records into MySQL.")

    except Error as e:
        print(f"Error inserting data into MySQL: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

if __name__ == "__main__":
    data = fetch_data_from_mongodb()

    if data:
        insert_data_into_mysql(data)
    else:
        print("No data to insert into MySQL.")